In [59]:
import os
import json
import pandas as pd

In [60]:
with open(os.path.join('drugbank_all_full_database.xml', 'one_drug_small_cured.json'), 'r') as file:
    drug_dict = json.load(file)


In [111]:
with open(os.path.join('drugbank_all_full_database.xml', 'one_drug_biotech_cured.json'), 'r') as file:
    biotech_dict = json.load(file)

In [112]:
with open(os.path.join('drugbank_all_full_database.xml', 'one_drug_mixed_cured.json'), 'r') as file:
    mixed_dict = json.load(file)

In [ ]:
with open(os.path.join('drugbank_all_full_database.xml', 'one_drug_mixed_cured.json'), 'r') as file:
    full_dict = json.load(file)

In [113]:
my_dict = {}
for key in drug_dict[0].keys():
    my_dict[key] = []
my_dict.pop('calculated-properties')
for property in drug_dict[0]['calculated-properties']['property']:
    my_dict[property['kind']] = []
my_dict['atc_code'] = []
my_dict['atc_code_$'] = []
for i in range(4):
    my_dict[f'atc_code_{i}'] = []
    my_dict[f'atc_code_{i}_$'] = []


str_and_nulls = []
for key,value in drug_dict[0].items():
    if type(value) in (str, float) or value == None: 
        str_and_nulls.append(key)

for mol in mixed_dict:
    if mol['@type'] != 'biotech':
        for key in my_dict.keys():
            if key in mol.keys():
                if key in str_and_nulls:
                    my_dict[key].append(mol[key])
                elif key == 'drugbank-id':
                    my_dict[key].append(mol[key][0]['$'])
                elif key == 'groups':
                    my_dict[key].append(mol[key]['group'][0])
                

            else:
                for inst in mol:
                    if inst == 'calculated-properties': 
                        property_kind_values = set()  
                        for property in mol['calculated-properties']['property']:
                            if property['kind'] not in property_kind_values:
                                property_kind_values.add(property['kind'])
                                if key == property['kind']:
                                    my_dict[key].append(property['value'])
                
                
                    elif inst == 'act-codes':
                        my_dict['atc_code'].append(value['atc-code'][0]['@code'])
                        my_dict['atc_code_$'].append(value['atc-code'][0]['$'])
                        for i, value in enumerate(key['atc-code'][0]['level']):
                            my_dict[f'act_code_{i}'] = value['atc-code'][0]['level'][i]['@code']
                            my_dict[f'act_code_{i}_$'] = value['atc-code'][0]['level'][i]['$']

                if key not in property_kind_values:
                    my_dict[key].append(None)

In [114]:
my_dict

{'@type': ['small molecule', 'small molecule'],
 '@created': ['2022-12-02', '2022-12-02'],
 '@updated': ['2022-12-03', '2022-12-13'],
 'drugbank-id': ['DB17297', 'DB17298'],
 'name': ['Enitociclib', 'Disufenton'],
 'cas-number': ['1610408-97-3', '168021-77-0'],
 'unii': ['1255AT22ZJ', '797K01YF1M'],
 'average-mass': [404.44, 337.36],
 'monoisotopic-mass': [404.111853336, 337.028994175],
 'groups': ['investigational', 'investigational'],
 'mixtures': [None, None],
 'manufacturers': [None, None],
 'atc-codes': [None, None],
 'experimental-properties': [None, None],
 'snp-effects': [None, None],
 'snp-adverse-drug-reactions': [None, None],
 'targets': [None, None],
 'enzymes': [None, None],
 'carriers': [None, None],
 'transporters': [None, None],
 'logP': ['3.71', '-2.1'],
 'logS': ['-4.7', '-2.9'],
 'Water Solubility': ['7.76e-03 g/l', '3.98e-01 g/l'],
 'IUPAC Name': ['(S)-[(2-{[5-fluoro-4-(4-fluoro-2-methoxyphenyl)pyridin-2-yl]amino}pyridin-4-yl)methyl](imino)methyl-lambda6-sulfanone',

In [117]:
lens= []
for key, value in my_dict.items():
    lens.append(len(value))
print(lens)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [118]:
drug_df = pd.DataFrame(my_dict)

In [119]:
drug_df

,@type,@created,@updated,drugbank-id,name,cas-number,unii,average-mass,monoisotopic-mass,groups,...,atc_code,atc_code_$,atc_code_0,atc_code_0_$,atc_code_1,atc_code_1_$,atc_code_2,atc_code_2_$,atc_code_3,atc_code_3_$
0,small molecule,2022-12-02,2022-12-03,DB17297,Enitociclib,1610408-97-3,1255AT22ZJ,404.44,404.111853,investigational,...,None,None,None,None,None,None,None,None,None,None
1,small molecule,2022-12-02,2022-12-13,DB17298,Disufenton,168021-77-0,797K01YF1M,337.36,337.028994,investigational,...,None,None,None,None,None,None,None,None,None,None


In [120]:
drug_df.to_csv(os.path.join('drugbank_all_full_database.xml', 'molecule_dataframe.csv'))